In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
#os.chdir("../Datastets")
#myData = pd.read_csv(r'C:\Users\Paul\Projects\Skill_Notebooks\Module_5\DataSets\mycar.csv')
myData = pd.read_csv(r'C:\Users\wangshu202040\Skill_Notes\Module_5\DataSets\mycar.csv')

In [ ]:
myData.head()

In [ ]:
myData.shape

In [ ]:
X = myData.iloc[:,:-1].values
Y = myData.iloc[:,1].values

In [ ]:
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.3)

In [ ]:
from sklearn.linear_model import LinearRegression
myModel = LinearRegression()
myModel.fit(X_train,Y_train)

In [ ]:
y_pred = myModel.predict(X_test)
y_pred

# <span style="color: orange; font-weight: bold; font-size:16pt">3A.5 Линейная регрессия. Предобработка</span>

In [ ]:
colors = ['#50248f', '#38d1ff']
sns.palplot(sns.color_palette(colors))

In [ ]:
data = pd.read_csv(r'C:/Users/wangshu202040/Skill_Notes/Module_5/DataSets/data_flats2.csv', sep = ';')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
cols = data.columns
fig, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colors))

У нас пропуски по сути есть только в одном признаке — жилой площади. Просто не будем брать её в модель.

Далее всегда есть смысл посмотреть на распределение нашей целевой переменной и понять, какие значения мы будем предсказывать:

In [ ]:
data.price_doc.hist();

У нашего распределения есть проблема — слишком сильный перепад. Много квартир в среднем сегменте, но очень мало дорогих квартир. На практике часто в таких случаях логорифмируют переменную, чтобы уменьшить перепады и сгладить хвост.

In [ ]:
data['price_doc'] = data['price_doc'].apply(lambda w: np.log(w+1))
data.price_doc.hist();

Теперь займемся отбором признаков.  Для начала нам надо проверить, нет ли мультиколлинеарности — сильной взаимосвязи между независимыми признаками. Для этого построим матрицу корреляций для признаков:

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize = (12,12))
sns.heatmap(data.corr(),square = True,
           annot = True, fmt = '.1f',linewidths=.1,cmap='RdBu')

In [ ]:
cols_to_del = ['id','preschool_education_centers_raion','kindergarten_km','park_km','kremlin_km','life_sq']

In [ ]:
data1 = data.drop(cols_to_del,axis=1)

In [ ]:
data1 = data1.dropna()

In [ ]:
# X = data1.iloc[:,:-1].values
# Y = data1.iloc[:,1].values

X = data1.drop(['price_doc'],axis = 1)
Y = data1['price_doc']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state=77)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
rc = RobustScaler()
X_train_tr = rc.fit_transform(X_train)
X_test_tr = rc.transform(X_test)

## Model

In [ ]:
from sklearn.linear_model import LinearRegression
myModel = LinearRegression()
myModel.fit(X_train_tr,Y_train)

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
Y_pred = myModel.predict(X_test_tr)
MSE = np.round(mse(np.exp(Y_test)-1,np.exp(Y_pred)-1),0)

In [ ]:
print(f'MSE = {MSE}')

# <span style="color: orange; font-weight: bold; font-size:16pt">3A.6. Линейная регрессия. Практика №1</span>

## 1. Линейная регрессия. Реализация

In [ ]:
data = load_boston()
data['data'].shape

### 1.1. Реализация линейной регрессии с использованием матричных операций

Линейная регрессия выражается следующей зависимостью:
$$y=X\theta+\epsilon,$$
где $X$ — матрица объекты-признаки, $y$ — вектор целевых значений, соответствующих $X$, $\theta$ — параметр линейной регрессии, $\epsilon$ — некоторый шум.

Из данного следует выражение для $\theta$ как:
$$X^Ty=X^TX\theta \rightarrow \theta=(X^TX)^{-1}X^Ty$$

Реализуем выражение для $\theta$ с помощью операций линейной алгебры библиотеки Numpy:

In [ ]:
def linreg_linear(X,y):
    theta = np.linalg.solve(X.T@X, X.T@y)
    return theta

In [ ]:
# Подготовить данные

X, y = data['data'], data['target']

X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

In [ ]:
# Вычислить параметр theta
theta = linreg_linear(X,y)

In [ ]:
theta.shape

In [ ]:
# Сделать предсказания для тренировочной выборки
y_pred = np.dot(X,theta)

In [ ]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true,y_pred)
    rmse = np.sqrt(mse)
    print (f'MSE = {mse:.2f}, RMSE= {rmse:.2f}')

In [ ]:
# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

In [ ]:
plt.hist(y);

In [ ]:
# plot data and model prediction
N=len(y)
plt.plot(np.arange(1,N+1),y,'bs-',label='Data')
plt.plot(np.arange(1,N+1),y_pred,'ro--',label='Model pred.')

plt.legend()
plt.show()

In [ ]:
# Разбить выборку на train/valid, вычислить theta,
# сделать предсказания и посчитать ошибки MSE и RMSE

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = linreg_linear(X_train, y_train)
y_pred = X_valid.dot(theta)
y_train_pred = X_train.dot(theta)

In [ ]:
print_regression_metrics(y_valid, y_pred)
print_regression_metrics(y_train, y_train_pred)

#### Задание 3.6.2

Постройте модель при помощи sklearn. Используйте параметры по умолчанию, обучите на всей выборке и посчитайте RMSE.
Ответ округлите до сотых. Пример ввода: 5.55.

In [ ]:
X, y = data['data'], data['target']

X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
print_regression_metrics(y,y_pred)

In [ ]:
df = pd.DataFrame(X)
A = df.describe().reset_index()
A.loc[A['index'] == 'std'].max()

 #### Задание 3.6.4

Обучите регрессию без дополнительного столбца единиц. Какой получился RMSE?
Ответ округлите до сотых. Пример ввода: 5.55.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Подготовить данные
X, y = data['data'], data['target']


In [ ]:
lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
print_regression_metrics(y,y_pred)

#### Задание 3.6.5

Очистите данные от строк, где значение признака  меньше . Какой получился RMSE?
Ответ округлите до сотых. Пример ввода: 5.55.

In [ ]:
X = pd.DataFrame(data = data['data'],columns=data['feature_names'])
y = data['target']
X['ones'] = 1

In [ ]:
y = y[X['B']>50]
X = X[X['B']>50]

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
print_regression_metrics(y,y_pred)

#### Задание 3.6.6

Нормализуйте признаки и обучите линейную регрессию матричным методом. Какой получился RMSE?
Ответ округлите до сотых. Пример ввода: 5.55.

In [ ]:
X = pd.DataFrame(data = data['data'],columns=data['feature_names'])
y = data['target']
X['ones'] = 1

In [ ]:
rc = RobustScaler()
X_train_tr = rc.fit_transform(X)

In [ ]:
# Вычислить параметр theta
theta = linreg_linear(X,y)

In [ ]:
y_pred = X@theta

In [ ]:
print_regression_metrics(y,y_pred)

### 1.2. Реализация линейной регрессии с использованием методов оптимизации

Для реализации линейной регрессии с помощью методов оптимизации будем использовать функцию ошибки **среднего квадратичного** ([Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error)), которая является выпуклой функцией в n-мерном пространстве $\mathbb{R}^n$ и в общем виде выглядит следующим образом:
$$MSE = \frac{1}{n} * \sum_{i=1}^{n}{(y_i - a(x_i))^2}.$$
Здесь $x_i$ — вектор-признак $i$-го объекта обучающей выборки, $y_i$ — истинное значение для $i$-го объекта, $a(x)$ — алгоритм, предсказывающий для данного объекта $x$ целевое значение (иначе говоря, предсказанное значение), $n$ — кол-во объектов в выборке.

В случае линейной регрессии $MSE$ представляется как:
$$MSE(X, y, \theta) = \frac{1}{2n} * \sum_{i=1}^{n}{(y_i - \theta^Tx_i)^2} = \frac{1}{2n} \lVert{y - X\theta}\rVert_{2}^{2}=\frac{1}{2n} (y - X\theta)^T(y - X\theta),$$
где $\theta$ — параметр модели линейной регрессии, $X$ — матрица объекты-признаки, $y$ - вектор истинных значений, соответствующих $X$.

Возьмем первый вариант представления функции ошибки и посчитаем ее градиент по параметру $\theta$, предварительно переименовав $MSE$ в $L$:
$$L=\frac{1}{2n} * \sum_{i=1}^{n}{(y_i - \theta^Tx_i)^2}$$
$$\nabla L = \frac{1}{n}\sum_{i=1}^{n}{(\theta^Tx_i - y_i) \cdot x_i} = \frac{1}{n}X^T(X\theta - y)$$

In [ ]:
# Реализовать функцию вычисления градиента функции MSE

def calc_mse_gradient(X, y, theta):
    n = X.shape[0]
    grad = 1. / n * X.transpose().dot(X.dot(theta) - y)
    
    return grad

In [ ]:
# Реализовать функцию, осуществляющую градиентный шаг
# (функция должна содержать параметр величины шага alpha - learning rate)

def gradient_step(theta, theta_grad, alpha):
    return theta - alpha * theta_grad

In [ ]:
# Реализовать функцию цикла градиентного спуска с доп. параметрами
# начального вектора theta и числа итераций

def optimize(X, y, grad_func, start_theta, alpha, n_iters):
    theta = start_theta.copy()
    
    for i in range(n_iters):
        theta_grad = grad_func(X, y, theta)
        theta = gradient_step(theta, theta_grad, alpha)
    
    return theta

In [ ]:
# Разбить таблицу данных на матрицы X и y
X, y = data['data'], data['target']

# Добавить фиктивный столбец единиц (bias линейной модели)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
m = X.shape[1]

In [ ]:
# Оптимизировать параметр линейной регрессии theta на всех данных
theta = optimize(X, y, calc_mse_gradient, np.ones(m), 0.001, 100)

In [ ]:
theta

In [ ]:
X.max(axis=0)

In [ ]:
print(data['feature_names'][np.argmax(X.std(axis=0)) + 1])
print(np.max(X.std(axis=0)))

In [ ]:
# Нормализовать даннные с помощью стандартной нормализации
X, y = data['data'], data['target']
X = (X - X.mean(axis=0)) / X.std(axis=0)

In [ ]:
# Добавить фиктивный столбец единиц (bias линейной модели)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
X.max(axis=0)

In [ ]:
theta = optimize(X, y, calc_mse_gradient, np.ones(m), 0.01, 5000)

In [ ]:
theta

In [ ]:
# Сделать предсказания при полученных параметрах
y_pred = X.dot(theta)

In [ ]:
# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

In [ ]:
# Разбить выборку на train/valid, оптимизировать theta,
# сделать предсказания и посчитать ошибки MSE и RMSE

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = optimize(X_train, y_train, calc_mse_gradient, np.ones(m), 0.01, 5000)
y_pred = X_valid.dot(theta)

print_regression_metrics(y_valid, y_pred)

# <span style="color: orange; font-weight: bold; font-size:16pt">3А.7. Линейная регрессия. Практика №2</span>

### 2.1.Когда использовать матричные операции вместо градиентного спуска в линейной регрессии

In [22]:
def print_regression_metrics(y_true, y_pred):
    '''Print the value of errors
    INPUT: target value true,
           Predicted value
    OUTPUT:Value of MSE and RMSE'''
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')


def prepare_boston_data():
    data = load_boston()
    X, y = data['data'], data['target']
    # Нормализовать даннные с помощью стандартной нормализации
    X = (X-X.mean(axis=0) / X.std(axis=0))
    # Добавить фиктивный столбец единиц (bias линейной модели)
    X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
    return X, y

Прежде чем начать, обернем написанную нами линейную регрессию методом матричных операций в класс:

In [23]:
class LinRegAlgebra():
    def __init__(self):
        self.theta = None

    def fit(self, X, y):
#         self.theta = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
        self.theta = np.linalg.solve(X.T@X, X.T@y)

    def predict(self, X):
        return X@self.theta

Проведем замеры скорости работы алгоритмов на матричных операциях и на градиентном спуске. Предварительно найдем параметры для метода, основанного на градиентном спуске, так, чтобы значения метрик максимально совпадало со значениями в случае первого алгоритма.

In [24]:
X, y = prepare_boston_data()

In [25]:
linreg_alg = LinRegAlgebra()
linreg_alg.fit(X,y)
y_pred = linreg_alg.predict(X)

# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y,y_pred)

MSE = 21.89, RMSE = 4.68


506